In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import cv2
import shutil
import torch
from pathlib import Path
import pandas as pd
import numpy as np

from helper_functions.tile import tile_img
from helper_functions.stitch import stitch_labels
from helper_functions.evaluate_helpers import yolo_format_to_bbox_corners, _evaluate, _precision_recall_f1

from ultralytics import YOLO

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/home/jupyter-william_dimaculang-7f3b4/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'.


In [2]:
def predict_yolo(pred_path:str,  # folder containing all images for inference")
                 label_save_path:str, # folder to save the label txts stitched
                 weight_dir:str, # path to pre-trained model weights
                 save_tiles_path:str="tiled", # temporary folder to store tiled images 
                 conf:float=0.25, # conf threshold of model
                 save_txt:bool=True):
    
    pred_path = Path(pred_path)
    label_save_path = Path(label_save_path)


    pred_imgs = list(pred_path.glob("*.jpg")) # get all images path in pred_path

  

    for img_path in pred_imgs:
        img = cv2.imread(str(img_path))
        HEIGHT, WIDTH = img.shape[:2]

        tile_img(img=img,
                save_path=save_tiles_path)


        # use this if CLI is not working
        device='cuda' if torch.cuda.is_available() else 'cpu'
        model = YOLO(weight_dir) # instantiate model
        
        model.predict(source=save_tiles_path, save_txt=save_txt, exist_ok=True, conf=conf)
        #model.predict(source=save_tiles_path, save_txt=save_txt, exist_ok=True, save_conf=False)



        if not label_save_path.exists():
            os.makedirs(label_save_path, exist_ok=True)

        master_df = stitch_labels("./runs/detect/predict/labels",
                      HEIGHT = HEIGHT,
                      WIDTH=WIDTH)

        master_df.to_csv(str(label_save_path / img_path.name[:-4])+".txt", sep=" ", header=None, index=False) # save stitched labels 
        
        shutil.rmtree("runs")

## test parameters setup

In [3]:
conf_thres_list = [0.2, 0.3, 0.4, 0.5, 0.6]

gt_location = 'leaf_flower_test'
#gt_location = 'tomatoes test'
gt_paths = list(Path(gt_location).glob("*.txt"))


gt_paths.sort()

# save path of detections in tiled imgs
tile_imgs_path = 'tiled'
output_path = './runs/detect/predict/labels' 
os.makedirs(output_path, exist_ok=True)

# where to save stitched pred outputs
test_set_pred_output = Path('./test_set_preds')
os.makedirs(test_set_pred_output, exist_ok=True)

df_col_names = ['class', 'x1', 'y1', 'w', 'h'] 

idx2class = {0:'flower', 1:'leaf'}
target_class_list = [0, 1] # idx list of obj classes

# idx2class = {0:'flower', 1:'leaf', 2: 'tomato'}
# target_class_list = [0, 1, 2] # idx list of obj classes


#model_name = 'yolov8l'
#weights_dir = f'yolov8_weights_notomato/{model_name}/best.pt'
#weights_dir = f'tomato_bed/{model_name}/weights/best.pt'


# path to store csv results
csv_path = './csv_test_results'

In [4]:
model_names = [path.stem for path in list(Path('yolov8_weights_notomato').glob('*')) if path.stem != '.ipynb_checkpoints']
#model_names = [path.stem for path in list(Path('tomato_bed').glob('*')) if path.stem != '.ipynb_checkpoints']

model_names = [name for name in model_names if name!='previous'] # remove 'previous' folder
model_names.sort() 
model_names

['yolov8l_flowerleaf_crop',
 'yolov8l_flowerleaf_noaug',
 'yolov8l_flowerleaf_rotate',
 'yolov8m_flowerleaf_crop',
 'yolov8m_flowerleaf_noaug',
 'yolov8m_flowerleaf_rotate',
 'yolov8s_flowerleaf_crop',
 'yolov8s_flowerleaf_noaug',
 'yolov8s_flowerleaf_rotate']

## test pipeline

In [9]:
# reset folders
shutil.rmtree('test_set_preds')
shutil.rmtree('tiled')

os.makedirs('test_set_preds')
os.makedirs('tiled')

In [10]:
# store here the prec, recall, and f1 of model using different confidence thres
precision_ave_list = []
recall_ave_list = []
f1_ave_list = []

conf_thres = conf_thres_list[3]
model_name = model_names[2]
weights_dir = f'yolov8_weights_notomato/{model_name}/weights/best.pt'


# input and output directories
input_path = Path(gt_location)

output_parent_dir = Path("test_set_preds")
os.makedirs(output_parent_dir, exist_ok=True) # make folder if it doesnt existEros Hacinas


print("Detecting objects...")
# model prediction
os.makedirs(output_parent_dir, exist_ok=True) # make output folder
predict_yolo(pred_path=str(input_path),
          label_save_path=str(output_parent_dir), 
          weight_dir=weights_dir, conf=conf_thres)

print("Detection done.")


Detecting objects...



image 1/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile1.jpg: 512x640 28 leafs, 14.0ms
image 2/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile2.jpg: 512x640 1 flower, 30 leafs, 13.3ms
image 3/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile3.jpg: 480x640 7 flowers, 14 leafs, 17.7ms
image 4/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile4.jpg: 512x640 27 leafs, 13.8ms
image 5/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile5.jpg: 512x640 25 leafs, 13.2ms
image 6/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile6.jpg: 480x640 8 flowers, 14 leafs, 14.1ms
image 7/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile7.jpg: 512x640 31 leafs, 13.8ms
image 8/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile8.jpg: 512x640 34 leafs, 13.2ms
image 9/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile9.jpg: 512x640 1 flower, 31 leafs, 13.2ms
Speed: 2.5ms preprocess, 14.0ms inference, 1.3ms postproc

image 5/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile5.jpg: 512x640 20 leafs, 13.2ms
image 6/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile6.jpg: 480x640 13 leafs, 14.3ms
image 7/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile7.jpg: 512x640 (no detections), 14.1ms
image 8/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile8.jpg: 512x640 (no detections), 13.2ms
image 9/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile9.jpg: 512x640 13 leafs, 16.1ms
Speed: 2.5ms preprocess, 14.0ms inference, 1.4ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict
7 labels saved to runs/detect/predict/labels

image 1/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile1.jpg: 512x640 10 leafs, 13.8ms
image 2/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile2.jpg: 512x640 20 leafs, 13.2ms
image 3/9 /home/jupyter-william_dimaculang-7f3b4/tomatoes/tiled/tile3.jpg: 480x640 22 leafs, 14.2ms
image 

Detection done.


In [11]:
# precision, recall, and f1
pred_paths = list(Path("test_set_preds").glob("*.txt")) # get paths of all stitched labels 
pred_paths.sort()
    
target_class = 0

precision_list = []
recall_list = []
f1_list = []

for gt_path, pred_path in zip(gt_paths, pred_paths):
    # make sure both txt files have the same name
    assert gt_path.name == pred_path.name, "[Error]: groundtruth and predicted txt files have different names"

    gt_df = pd.read_csv(str(gt_path), sep=' ', names=df_col_names, index_col=False) # open gt txt file
    pred_df = pd.read_csv(str(pred_path), sep=' ', names=df_col_names, index_col=False) # open pred txt file


    # get tp, fp, and fn of target class
    TP, FP, FN = _evaluate(gt_df, pred_df, target_class=target_class)

    # perform evaluation if tp or (fp and fn) is not zero; this prevents dividing by zero 
    if TP > 0 | (FP > 0 & FN > 0):
        prec, recall, f1 = _precision_recall_f1(TP, FP, FN)

        precision_list.append(prec)
        recall_list.append(recall)
        f1_list.append(f1)

# get average of precision, recall, and f1 if lists are not empty
precision_ave, recall_ave, f1_ave = np.mean(precision_list), np.mean(recall_list), np.mean(f1_list)
#print(f'{idx2class[target_class]} metrics @ conf: {conf_thres}')
print(f'precision: {precision_ave:.2f}, recall: {recall_ave:.2f}, f1: {f1_ave:.2f}\n')


# reset folders
#shutil.rmtree('test_set_preds')
#shutil.rmtree('tiled')

#os.makedirs('test_set_preds')
#os.makedirs('tiled')

precision: 0.92, recall: 0.88, f1: 0.89

